In [1]:
import requests
import re
import pandas as pd
import configparser

# PTX API
from wsgiref.handlers import format_date_time
from time import mktime
from datetime import datetime
import hmac
from hashlib import sha1
import base64
from requests import request
import json
from pprint import pprint

# AWS
import boto3
import awswrangler as wr

In [2]:
config = configparser.ConfigParser()
config.read('credentials.cfg')
#config.items('PTX')

['credentials.cfg']

In [3]:
#function
class Auth():

    def __init__(self, app_id, app_key):
        self.app_id = app_id
        self.app_key = app_key

    def get_auth_header(self):
        xdate = format_date_time(mktime(datetime.now().timetuple()))
        hashed = hmac.new(self.app_key.encode('utf8'), ('x-date: ' + xdate).encode('utf8'), sha1)
        signature = base64.b64encode(hashed.digest()).decode()

        authorization = 'hmac username="' + self.app_id + '", ' + \
                        'algorithm="hmac-sha1", ' + \
                        'headers="x-date", ' + \
                        'signature="' + signature + '"'
        return {
            'Authorization': authorization,
            'x-date': format_date_time(mktime(datetime.now().timetuple())),
            'Accept - Encoding': 'gzip'
        }
    

def get_mrt_traffic_data_link():
    
    import ssl
    ssl._create_default_https_context = ssl._create_unverified_context    
    
    url = 'https://data.taipei/api/dataset/' \
      '63f31c7e-7fc3-418b-bd82-b95158755b4d' \
      '/resource/eb481f58-1238-4cff-8caa-fa7bb20cb4f4/download'

    # 設定每個cell可顯示字串長度
    # default = 50
    pd.options.display.max_colwidth = 400
    return pd.read_csv(url)
    
    
def get_ptx_data(service):
    station_url = f'https://ptx.transportdata.tw/MOTC/v2/' \
                  f'Rail/Metro/{service}/TRTC?%24format=JSON'

    a = Auth(config['PTX']['PTX_APP_ID'], config['PTX']['PTX_APP_KEY'])
    response = request('get', station_url, headers= a.get_auth_header())
    my_json = response.content.decode('utf-8')

    return json.loads(my_json)


def json_to_df(json_data):
    return pd.json_normalize(json_data, sep='_')


def station_key_generator(row):
    if (row["StationID"] == "BL07") & (row["StationName_Zh_tw"] == "板橋"):
        return "BL板橋"
    elif (row["StationID"] == "Y16") & (row["StationName_Zh_tw"] == "板橋"):
        return "Y板橋"
    elif (row["StationID"] == "O12") & (row["StationName_Zh_tw"] == "大橋頭"):
        return "大橋頭站"
    else:
        return row["StationName_Zh_tw"]


# Set up s3 connection and bucktet

## set up bucket

In [4]:
s3 = boto3.resource('s3'
                    , region_name=config['AWS']['AWS_DEFAULT_REGION']
                    , aws_access_key_id=config['AWS']['ACCESS_KEY']
                    , aws_secret_access_key=config['AWS']['SECRET_ACCESS_KEY'])


#s3.create_bucket(Bucket='my-bucket')
if s3.Bucket(config['AWS']['BUCKET_NAME']).creation_date is None:
    s3.create_bucket(Bucket=config['AWS']['BUCKET_NAME'],
                     CreateBucketConfiguration={'LocationConstraint': config['AWS']['AWS_DEFAULT_REGION']} )
    print("Bucket created")
else:
    print("Bucket exists")


# Print out bucket names
for bucket in s3.buckets.all():
    print(bucket.name)

Bucket exists
mrt-traffic


## set up session

In [7]:
%%time

session = boto3.Session(region_name=config['AWS']['AWS_DEFAULT_REGION']
                        , aws_access_key_id=config['AWS']['ACCESS_KEY']
                        , aws_secret_access_key=config['AWS']['SECRET_ACCESS_KEY'])

folder_name = "staging-data"


CPU times: user 17.9 ms, sys: 1.94 ms, total: 19.8 ms
Wall time: 22.5 ms


# Get datasets 

## dataset 1: MRT hourly traffic data

In [8]:
df_source = get_mrt_traffic_data_link()
df_source.tail(5)

,年月,資料路徑
58,202111,http://tcgmetro.blob.core.windows.net/stationod/%E8%87%BA%E5%8C%97%E6%8D%B7%E9%81%8B%E6%AF%8F%E6%97%A5%E5%88%86%E6%99%82%E5%90%84%E7%AB%99OD%E6%B5%81%E9%87%8F%E7%B5%B1%E8%A8%88%E8%B3%87%E6%96%99_202111.csv
59,202112,http://tcgmetro.blob.core.windows.net/stationod/%E8%87%BA%E5%8C%97%E6%8D%B7%E9%81%8B%E6%AF%8F%E6%97%A5%E5%88%86%E6%99%82%E5%90%84%E7%AB%99OD%E6%B5%81%E9%87%8F%E7%B5%B1%E8%A8%88%E8%B3%87%E6%96%99_202112.csv
60,202201,http://tcgmetro.blob.core.windows.net/stationod/%E8%87%BA%E5%8C%97%E6%8D%B7%E9%81%8B%E6%AF%8F%E6%97%A5%E5%88%86%E6%99%82%E5%90%84%E7%AB%99OD%E6%B5%81%E9%87%8F%E7%B5%B1%E8%A8%88%E8%B3%87%E6%96%99_202201.csv
61,202202,http://tcgmetro.blob.core.windows.net/stationod/%E8%87%BA%E5%8C%97%E6%8D%B7%E9%81%8B%E6%AF%8F%E6%97%A5%E5%88%86%E6%99%82%E5%90%84%E7%AB%99OD%E6%B5%81%E9%87%8F%E7%B5%B1%E8%A8%88%E8%B3%87%E6%96%99_202202.csv
62,202203,http://tcgmetro.blob.core.windows.net/stationod/%E8%87%BA%E5%8C%97%E6%8D%B7%E9%81%8B%E6%AF%8F%E6%97%A5%E5%88%86%E6%99%82%E5%90%84%E7%AB%99OD%E6%B5%81%E9%87%8F%E7%B5%B1%E8%A8%88%E8%B3%87%E6%96%99_202203.csv


In [9]:
%%time

df_source = get_mrt_traffic_data_link()

file_path = f's3://{config["AWS"]["BUCKET_NAME"]}/{folder_name}/traffic'

for idx, row in df_source.iterrows():
    if row[0] >= 202201:
        print(f"processing mrt traffic data {row[0]}...")
        df_traffic = pd.read_csv(row[1])
        
        # get traffic of first n days only
        df_traffic_test = df_traffic[df_traffic["日期"].str[-2:]<="03"].copy()
        
        wr.s3.to_csv(df_traffic_test, 
                     f'{file_path}/mrt_traffic_{row[0]}.csv', 
                     index=False,
                     boto3_session=session)
       


processing mrt traffic data 202201...
processing mrt traffic data 202202...
processing mrt traffic data 202203...
CPU times: user 47.6 s, sys: 17.8 s, total: 1min 5s
Wall time: 8min 41s


## dataset 2: station id and name

In [53]:
json_station = get_ptx_data(service="Station")
df_station = json_to_df(json_station)
df_station = df_station[["StationID"
                         , "StationAddress"
                         , "BikeAllowOnHoliday"
                         , "LocationCity"
                         , "LocationCityCode"
                         , "StationName_Zh_tw"
                         , "StationName_En"
                         , "StationPosition_PositionLon"
                         , "StationPosition_PositionLat"
                         , "StationPosition_GeoHash"
                         , "VersionID"]]

df_station["station_join_key"] = df_station.apply(lambda row: station_key_generator(row), axis=1)
station_version_id = df_station["VersionID"].unique()[0]

wr.s3.to_csv(df_station, 
             f's3://{config["AWS"]["BUCKET_NAME"]}/{folder_name}/mrt_station_v{station_version_id}.csv', 
             index=False,
             boto3_session=session)
df_station.head(3)

,StationID,StationAddress,BikeAllowOnHoliday,LocationCity,LocationCityCode,StationName_Zh_tw,StationName_En,StationPosition_PositionLon,StationPosition_PositionLat,StationPosition_GeoHash,VersionID,station_join_key
0,BL01,236040新北市土城區中央路4段51之6號B3,True,新北市,NWT,頂埔,Dingpu,121.42050,24.96012,wsqmfzzen,4,頂埔
1,BL02,236036新北市土城區中央路3段105號B1,True,新北市,NWT,永寧,Yongning,121.43613,24.96682,wsqq53546,4,永寧
2,BL03,236017新北市土城區金城路1段105號B1,True,新北市,NWT,土城,Tucheng,121.44432,24.97313,wsqq5d1p1,4,土城


## dataset 3: exit id and name

In [54]:
json_exit = get_ptx_data(service="StationExit")
df_exit = json_to_df(json_exit)
df_exit = df_exit[["StationID","ExitID"
                   , "Stair"
                   , "Escalator"
                   , "Elevator"
                   , "StationName_Zh_tw"
                   , "StationName_En"
                   , "ExitName_Zh_tw"
                   , "ExitName_En"
                   , "ExitPosition_PositionLon"
                   , "ExitPosition_PositionLat"
                   , "ExitPosition_GeoHash"
                   , "VersionID"]]
df_exit["station_join_key"] = df_exit.apply(lambda row: station_key_generator(row), axis=1)
exit_version_id = df_exit["VersionID"].unique()[0]
wr.s3.to_csv(df_exit, 
             f's3://{config["AWS"]["BUCKET_NAME"]}/{folder_name}/mrt_exit_v{exit_version_id}.csv', 
             index=False,
             boto3_session=session)
df_exit.head()

,StationID,ExitID,Stair,Escalator,Elevator,StationName_Zh_tw,StationName_En,ExitName_Zh_tw,ExitName_En,ExitPosition_PositionLon,ExitPosition_PositionLat,ExitPosition_GeoHash,VersionID,station_join_key
0,BL01,1,True,2,True,頂埔,Dingpu,頂埔站出口1,Dingpu Exit 1,121.418218,24.959306,wsqmfzwn3,3,頂埔
1,BL01,2,True,2,False,頂埔,Dingpu,頂埔站出口2,Dingpu Exit 2,121.419000,24.959310,wsqmfzwwe,3,頂埔
2,BL01,3,True,1,True,頂埔,Dingpu,頂埔站出口3,Dingpu Exit 3,121.419600,24.959620,wsqmfzz03,3,頂埔
3,BL01,4,True,0,True,頂埔,Dingpu,頂埔站出口4,Dingpu Exit 4,121.420100,24.960390,wsqmfzzkv,3,頂埔
4,BL02,1,True,1,True,永寧,Yongning,永寧站出口1,Yongning Exit 1,121.435254,24.966714,wsqq5343w,3,永寧


## check again

In [14]:
traffic_station_name = df_traffic[["進站"]].drop_duplicates()
station_station_name = df_station[["station_join_key"]]
exit_station_name = df_station[["station_join_key"]].drop_duplicates()

display(
    traffic_station_name.merge(station_station_name,
                               left_on="進站",
                               right_on="station_join_key",
                               indicator = True, 
                               how='left').loc[lambda x : x['_merge']!='both']

    , traffic_station_name.merge(exit_station_name,
                                 left_on="進站",
                                 right_on="station_join_key",
                                 indicator = True, 
                                 how='left').loc[lambda x : x['_merge']!='both']
)

NameError: name 'df_traffic' is not defined

## read data from s3 bucket

In [41]:
df = wr.s3.read_csv(f's3://{config["AWS"]["BUCKET_NAME"]}/{folder_name}/mrt_exit.csv',
               boto3_session=session)

In [42]:
df

,StationID,ExitID,Stair,Escalator,Elevator,StationName_Zh_tw,StationName_En,ExitName_Zh_tw,ExitName_En,ExitPosition_PositionLon,ExitPosition_PositionLat,ExitPosition_GeoHash,VersionID,station_join_key
0,BL01,1.0,True,2,True,頂埔,Dingpu,頂埔站出口1,Dingpu Exit 1,121.418218,24.959306,wsqmfzwn3,3,頂埔
1,BL01,2.0,True,2,False,頂埔,Dingpu,頂埔站出口2,Dingpu Exit 2,121.419000,24.959310,wsqmfzwwe,3,頂埔
2,BL01,3.0,True,1,True,頂埔,Dingpu,頂埔站出口3,Dingpu Exit 3,121.419600,24.959620,wsqmfzz03,3,頂埔
3,BL01,4.0,True,0,True,頂埔,Dingpu,頂埔站出口4,Dingpu Exit 4,121.420100,24.960390,wsqmfzzkv,3,頂埔
4,BL02,1.0,True,1,True,永寧,Yongning,永寧站出口1,Yongning Exit 1,121.435254,24.966714,wsqq5343w,3,永寧
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456,Y18,3.0,True,2,False,頭前庄,Touqianzhuang,頭前庄站出口3,Touqianzhuang Exit 3,121.462247,25.039853,wsqq7yq5e,3,頭前庄
457,Y18,4.0,True,2,False,頭前庄,Touqianzhuang,頭前庄站出口4,Touqianzhuang Exit 4,121.462242,25.040485,wsqq7yqp7,3,頭前庄
458,Y19,1.0,True,1,True,幸福,Xingfu,幸福站出口1,Xingfu Exit 1,121.460146,25.050126,wsqqebhxd,3,幸福
459,Y19,2.0,True,0,False,幸福,Xingfu,幸福站出口2,Xingfu Exit 2,121.460593,25.049545,wsqqebhuh,3,幸福


In [56]:
# 202203: 892143
# 202202: 892143
# 202201: 

df_t = wr.s3.read_csv(f's3://{config["AWS"]["BUCKET_NAME"]}/{folder_name}/traffic/mrt_traffic_202201.csv',
               boto3_session=session)
df_t.shape

(877982, 5)

In [40]:
df_t[(df_t['日期']=='2022-03-01') & (df_t['時段']==12) & (df_t['進站']=='松山機場')]

,日期,時段,進站,出站,人次
127449,2022-03-01,12,松山機場,松山機場,0
127450,2022-03-01,12,松山機場,中山國中,0
127451,2022-03-01,12,松山機場,南京復興,6
127452,2022-03-01,12,松山機場,忠孝復興,7
127453,2022-03-01,12,松山機場,大安,1
...,...,...,...,...,...
127563,2022-03-01,12,松山機場,板新,0
127564,2022-03-01,12,松山機場,Y板橋,0
127565,2022-03-01,12,松山機場,新埔民生,0
127566,2022-03-01,12,松山機場,幸福,0
